In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

str1 = "http://www.loopnet.com/for-lease/"
str2 = "/?sk=37743a9aa7a944b924909a9b3db5e4f4&bb=jqz4jqtsyHom9__ylB"

price = open("nyc_price_address.csv", "w")

price.write("address")
price.write(",")
price.write("price")
price.write(",")
price.write("zipcode")
price.write("\n")
history = {"result"}
for i in range(1, 25):
    strcur = str1 + str(i) + str2
    response = requests.get(strcur)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    el1 = soup.find_all(itemprop="streetAddress")
    el2 = soup.find_all(class_="listing-price")
    el3 = soup.find_all(itemprop="postalCode")
    
    for j in range(len(el1)):
        add = el1[j].text
        zipcode = el3[j].text
        pri = el2[j].text
        price.write(add)
        price.write(",")
        if pri != "Rent Not Disclosed":
            pri = pri.split("/")[0].rstrip()
            price.write(pri)
        else:
            price.write("NA")
        price.write(",")
        price.write(zipcode)
        price.write("\n")
        print("address is: " + add)
        print("price is: " + pri)
        history.add(add)

price.close()

address is: 825 Seventh Ave
price is: Rent Not Disclosed
address is: 4511 34th Ave
price is: $52.80 USD
address is: 4509 Broadway
price is: Rent Not Disclosed
address is: 992 5th Ave
price is: $62.40 USD
address is: 16 Court St
price is: $55.00 USD
address is: 161 6th Ave
price is: Rent Not Disclosed
address is: 49-02 Queens Blvd
price is: $40.00 USD
address is: 825 Seventh Ave
price is: Rent Not Disclosed
address is: 54 Thompson St
price is: Rent Not Disclosed
address is: 156 Bedford Ave
price is: $166.00 - $166.67 USD
address is: 42-07 Broadway
price is: $93.33 USD
address is: 356-362 Getty Ave
price is: $9.50 USD
address is: 724-730 Garrison Ave
price is: $15.25 USD
address is: 455 Jefferson St
price is: Rent Not Disclosed
address is: 433 Murray Hill Pky
price is: $8.50 USD
address is: 275 Seventh Ave
price is: Rent Not Disclosed
address is: 14 Wall St
price is: Rent Not Disclosed
address is: 199 Cook St
price is: $37.50 - $51.43 USD
address is: 1330 Avenue Of The Americas
price is:

In [43]:
price = pd.read_csv("nyc_price_address.csv")

### Remove all the NA data, transform price to float, and transform address to lat and lng.

In [38]:
import re

lat = []
lng = []
valid_address = []
city = " New York, NY, "
price_list = []
geolocator = Nominatim()
for index, row in price.iterrows():
    if type(row['price']) == float:
        continue
    else:
        address = row['address']
        location = geolocator.geocode(row['address'] + city + str(row['zipcode']))
        if location:
            lat.append(location[-1][0])
            lng.append(location[-1][1])
            valid_address.append(row['address'])
            pri = re.findall("\d+\.\d+", row['price'])[-1]
            print(pri)
            price_list.append(pri)


52.80
62.40
55.00
40.00
166.67
93.33
15.25
51.43
172.80
21.50
22.00
59.05
168.00
272.90
45.00
50.00
49.41
8.50
75.00
28.00
76.00
42.00
20.00
68.57
95.00
47.00
27.55
23.00
33.00
144.00
455.69
28.00
25.00
25.00
175.00
120.00
17.75
36.00
66.00
26.40
24.50
48.00
68.40
19.50
65.00
49.00
13.20
30.00
38.00
58.00
49.00
144.00
96.00
18.00
18.00
52.00
30.00
125.00
75.00
55.00
75.00
48.00
64.00
65.00
65.00
58.00
55.00
75.00
65.00
55.00
300.00
65.00
59.00
52.00
416.67
48.00
9.60
36.00
220.00
72.72
38.83
116.42
22.50
48.00
30.00
35.00
45.00
45.00
7.50
55.86
55.08
62.00
73.00
54.28
47.60
60.00
48.00
24.00
150.00
51.69
25.00
79.80
20.00
36.00
80.00
40.00
160.00
22.50
5.75
35.00
25.00
20.80
30.00
22.00
144.00
22.22
45.00
25.00
54.00
59.00
205.63
30.00
38.33
21.33
20.00
20.00
44.57
40.00
120.00
23.54
55.00
22.00
38.18
40.00
36.00
95.00
34.80


GeocoderServiceError: [Errno 65] No route to host

In [39]:
price_lat_lng = pd.DataFrame(index = range(len(lat)))

In [40]:
price_lat_lng['address'] = valid_address
price_lat_lng['lat'] = lat
price_lat_lng['lng'] = lng
price_lat_lng['price'] = price_list


In [42]:
price_lat_lng.to_csv("nyc_price_lat_lng.csv")